## 手写数字识别

---

#### 介绍

本实验将会介绍一个非常经典的数据集：MNIST 数据集，并且分析它的数据特征。然后我们将利用我们上一个所实现的神经网络类建立一个神经网络结构，对这个数据集继续学习，然后得到一个较好的神经网络结构。最后，我们将讨论整个神经网络的模型的优化方向，以及一些参数对模型准确率的影响。

#### 知识点

- MNIST 数据的介绍
- 数据的预处理
- 神经网络模型的训练
- 神经网络模型的预测
- 神经网络模型的优化

---

### 手写数字的数据集 MNIST

#### 数据的加载

识别笔记这个问题相对复杂，也非常模糊，因此这是一种检验人工智能的理想挑战。这不像进行大量数字相乘那样明确清晰。

这里有一套流行的手写数字图片来测试人工智能研究者的最新思想和算法。这套图片众所周知，非常流行。很多研究者在对比自己的算法的优缺点时，就会找同样利用该数据集的其它算法进行比较。

这个数据集称为手写数字的 MNIST 数据库。从受人尊敬的神经网络研究员 Yann LeCun 的 [网站](http://yann.lecun.com/exdb/mnist/) ，可以得到这个数据集。 


这个网页也列出了在学习和正确识别这些手写字符的方法以及这些方法的效果。我们将会多次提到这个列表，看看比起专业人士我们的方法表现如何。MNIST 数据库的格式不容易使用，因此其他人已经创建了相对简单的数据文件格式，参见 [该网站](http://pjreddie.com/projects/mnist-in-csv/) ，这对我们非常有帮助。 

这些文件称为 CSV 文件，这意味着纯文本中的每一个值都是由逗号分隔的。你可以轻松地在任何文本编辑器中查看这些数值，大部分的电子表格和数据分析软件也兼容 CSV 文件，它们是非常通用的标准。

这个网站提供了两个 CSV 文件：

- [训练集](http://www.pjreddie.com/media/files/mnist_train.csv)
- [测试集](http://www.pjreddie.com/media/files/mnist_test.csv)

顾名思义，训练集是用来训练神经网模型，优化神经网络的权重。每条样本既有输入，又有它所对应的输出。

测试集用于测试模型的优劣，判断算法的好坏。

将训练和测试数据集分开的想法，是为了确保可以使用神经网络之前没有见过的数据进行测试。如果把所有数据都放进去训练，然后再把这些数据中的一部分数据拿出来测试。那么，有可能神经网络的正确率会很高。但是很高的原因，并不是因为它训练的有多好，有可能只是因为它见过这个样本，知道他的正确输出。

让我们来一窥这些文件。下面显示的是 MNIST 测试集中的部分内容。

<img width = "600px" src="https://doc.shiyanlou.com/courses/uid1166617-20191220-1576826265443/wm">

哇！这看起来好像出事了！就像在 20 世纪 80 年代的电影中一样，计算机被黑客攻击了。

其实每一段，在 CSV 中是一行数据，知识这里显示不完全，所以出现了自动换行。

其实，这些记录或这些行的内容很容易理解：

- 每行数据的第一个值是标签，即这张图片表示的数字。如上图的两个条数据，一条表示的是 7 ，另外一条表示的是 2 。 
- 随后的值，由逗号分隔，是手写体数字的像素值。像素数组的尺寸是 $28 \times 28$。因此每行数据的标签后都有 $784$ 个值。如果想知道这里是否有 784 个值，你可以一个一个地数。 

但是，从这个长达 784 个值的列表中，人们很难看出这些数字组成了某人手写数字 7 的图片。之后，我们会将这些数字绘制为图像，让读者确认这 784 个值真的是手写数字的像素值。

在深入研究之前，让我们先下载 MINIST 数据集中，较小的子集文件。MNIST 数据的数据文件是相当大的，而较小的子集意味着我们可以实验、尝试和开发代码，而不会由于大量的数据集而拖慢计算机的速度，因此小数据集还是大有裨益的。一旦确定了乐于使用的算法和代码，我们就可以使用完整的数据集。

运行下面代码，下载较小的数据子集：

In [ ]:
!wget -nc https://labfile.oss.aliyuncs.com/courses/1500/minist-data.zip
!unzip -o "minist-data.zip" 

In [ ]:
!ls

接下来，我们使用 Python 来打开文件并获取其中的内容。代码如下： 

In [ ]:
#打开文件
data_file = open("mnist_train_100.csv", 'r') 
#对文件进行操作
data_list = data_file.readlines() 
#关闭文件
data_file.close() 

这里只有 3 行代码。让我们一一进行讨论。

第一行使用 open() 函数打开一个文件，参数列表如下：


- 第一个参数是文件的名称。其实，这不仅仅是文件名 “mnist_train_100.csv”，这是整个路径，其中包括了文件所在的目录。
- 第二个参数是可选的，它只是告诉 Python 我们希望如何处理文件。“r” 告诉 Python 以只读而不可写的方式打开文件。这样可以避免任何更改数据或删除数据的意外。


变量 data_file 是 open() 函数返回的一个文件句柄，也就是一个文件相关的对象。该对象里面存了很多关于文件的从操作，如读取文件等。

第二行代码，就是利用打开的 data_file 对象中的 readlines() 函数，将文件中的所有行读入变量 data_list 。这个变量包含了一个列表，列表中的一项表示着文件中的一行字符串。

顺便说一句，由于 readlines() 会将整个文件读取到内存中，因此你可能会听到别人告诉你不要使用这种方法。他们会告诉你，一次读取一行，对这行进行所需要进行的操作，然后移动到下一行。他们都没有错，不要将整个文件读入内存中，而是一次在一行上工作，这更有效率。但是，我们的文件不是很大，如果使用 readlines() ，那么代码相对容易一些。而对我们而言，在学习 Python 时简单和清晰是很重要的。 

第三行，在用完像文件这样的资源后，关闭和清理文件是一种很好的做法。如果不这样做，文件依然开着，这可能会造成问题。什么问题呢？有些程序可能不希望写入处在打开状态的文件，以免导致不一致。但是如果另一个人，明明已经用完了这个文件，却始终不放手，那么这个文件就始终没法写入其他东西。

接下来，让我们来查看一下从文件中读取的数据：

In [ ]:
print(len(data_list)) # 输出 datalist 的长度
print(data_list[0]) # 输出第一条数据

由上面结果可以看出：

- 该文件共有 100 条样本
- 第一个样本的实际标签为 5 ，其余的 784 个数字为每个像素点的颜色值。且这些颜色值的范围为：0~255

#### 数据的可视化

接下来，让我们把这一条数据转换成一张图片，看看是否真的是一张手写的 5 。

首先，我们需要导入 Python 的拓展库，用以使用数组和绘图。

In [ ]:
#导入 Numpy 进行数组相关操作
import numpy
#导入 matplotlib.pyplot 用于绘图操作。
import matplotlib.pyplot 
%matplotlib inline

那么接下来，我们需要做的事情，就是把上面的 `data_list[0]`  转换为一张图片，那么怎么转换呢？基本步骤如下：

- 以逗号为分割符，将 `data_list[0]` 切割成一个新的数组，取名为 `all_values`，数组大小为 1 + 784。

  - `all_values[0]`：表示 all_values 的第 0 个值，也就是该条数据的标签（数组的下标从 0 开始）。
  
  - `all_values[1:]`：表示 all_values 从 0 到最后的所有值，一共 784 个数
- 将 all_values[1:] 中的 784 个数，重新排列，排成（28,28）的矩阵
- 这个矩阵就是手写字符的原图

利用代码实现上面的步骤：

In [ ]:
all_values = data_list[0].split(',')
# 这里的 numpy.asfarray 是将 all_values[1:] 变成一个新的数组，原来 all_values [1:] 只是 all_values 的一部分
# 通过 asfarray 后就变成了一个完整的数组，再通过 reshape 重新排列,得到 (28,28) 的矩阵
image_array = numpy.asfarray(all_values[1:]).reshape((28, 28))
# 使用 imshow() 函数绘出 image_array。选择灰度调色板——cmap=“Greys（灰度）”，以更好地显示手写字符。
matplotlib.pyplot.imshow(image_array, cmap='Greys', interpolation='None')

#### 数据的预处理

我们已经知道如何获取和拆开 MNIST 数据文件中的数据，从而理解并可视化这些数据。接下来我们将对这些数据进行预处理，使神经网络的工作更加出色。

我们先前看到，如果输入数据和输出值，形状正好适合，这样它们就可以待在网络节点激活函数的舒适区域内，那么神经网络的工作会更出色。

我们需要做的第一件事情是将输入颜色值从较大的 0 到 255 的范围，缩放至较小的 0.01 到 1.0 的范围。我们刻意选择 0.01作为范围最低点，是为了避免先前观察到的 0 值输入最终会人为地造成权重更新失败。我们没有选择 0.99 作为输入的上限值，是因为输入 1.0 不会造成任何问题。

将在 0 到 255 范围内的原始输入值除以 255 ，就可以得到 0 到 1 范围的输入值。

然后，需要将所得到的输入乘以 0.99 ，把它们的范围变成 0.0 到 0.99。接下来，加上 0.01，将这些值整体偏移到所需的范围 0.01 到 1.00。下面的Python代码演示了这些操作： 

In [ ]:
#对所有输入数据进行范围缩小，方便神经网络计算
scaled_input = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
print(scaled_input)

#### 神经网络的输出

我们已经通过缩放和移位让 MNIST 数据准备就绪，可以输入神经网络进行训练和查询了。

现在，我们需要思考神经网络的输出。我们使用的逻辑函数不能输出如 -2.0 或 255 这样的数字，能输出的范围为 0.0 到 1.0，事实上不能达到 0.0 或 1.0，这是逻辑函数的极限值，逻辑函数仅接近这两个极限，但不能真正到达那里。因此，看起来在训练时必须调整目标值。

但是，实际上，我们要问自己一个更深层次的问题。输出应该是什么样子的？

如果退后一步，想想，我们要求神经网络做什么？我们会意识到，要求神经网络对图像进行分类，分配正确的标签。这些标签是   0 到 9 共 10 个数字中的一个。这意味着神经网络应该有 10 个输出层节点，每个节点对应一个可能的答案或标签。如果答案是“0”，输出层第一个节点激发，而其余的输出节点则保持抑制状态。如果答案是“9”，输出层的最后节点会激发，而其余的输出节点则保持抑制状态。下图详细阐释了这个方案，并显示了一些示例输出。 

<img width = "600px" src="https://doc.shiyanlou.com/courses/uid1166617-20191227-1577408360201/wm">

第一个示例是神经网络认为它看到的是数字 “5” 。可以看到，从输出层出现的最大信号来自于标签为 5 的节点。由于从标签 0 开始，因此这是第六个节点。这很容易吧。其余的输出节点产生的信号非常小，接近于零。舍入误差可能导致零输出，但事实上，要记住激活函数不会产生实际为零的输出。

下一个示例演示了如果神经网络认为它看到了手写的 “0” 将会发生的事情。同样，目前最大输出来自于第一个输出节点，对应的标签为 “0” 。

最后一个示例更有趣。这里，神经网络的最大输出信号来自于最后一个节点，对应标签 “9” 。然而，在标签为 “4” 的节点处，它得到了中等大小的输出。通常，我们会使用最大信号为答案，但是，可以看看网络为何会认为答案可能是“4”。也许是因为这张图片的笔迹使得它难以确定。

这真是太棒了！现在，我们需要把这些想法转换成目标数组，用于神经网络的训练。

如果训练样本的标签为 “5” ，那么需要创建输出节点的目标数组，其中除了对应于标签 “5” 的节点，其他所有节点的值应该都很小，这个数组看起来可能如 [0，0，0，0，0，1，0，0，0，0]。

事实上，我们已经明白了，试图让神经网络生成 0 和 1 的输出，对于激活函数而言是不可能的，这会导致大的权重和饱和网络，因此需要重新调整这些数字。我们将使用值 0.01 和 0.99 来代替 0 和 1 ，这样标签为 “5” 的目标输出数组为 [0.01, 0.01, 0.01, 0.01, 0.01, 0.99, 0.01, 0.01, 0.01, 0.01]。

仔细观察下列的 Python 代码，这些代码构建了目标矩阵：

In [ ]:
# 输出有 10 个节点
onodes = 10
# 使用 numpy.zeros()，创建用零填充的数组。加上了 0.01 ，解决刚才谈到的 0 输入造成的问题。
targets = numpy.zeros(onodes) + 0.01
# 下一行代码获得了 MNIST 数据集记录中的第一个元素，将其从字符串形式转换为整数形式。
#再将目标列表的正确元素设置为 0.99 
targets[int(all_values[0])] = 0.99
print(targets)

太好了，现在，用于训练和查询的输入数据以及输出数据已经准备齐全。接下来让我们来完整的对神经网络进行编写。

### 神经网络模型

在搭建神经网络之前，我们需要先把上一实验中，编写好的神经网络类给复制过来，方便后面我们的使用。

In [ ]:
#导入所需要的工具包
import numpy
import scipy.special
import matplotlib.pyplot
%matplotlib inline

#神经网络类的定义
#详细注释，请查看上一章节
class neuralNetwork:
    
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
    
        self.wih = numpy.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes, self.inodes))
        self.who = numpy.random.normal(0.0, pow(self.hnodes, -0.5), (self.onodes, self.hnodes))

        self.lr = learningrate
        
        self.activation_function = lambda x: scipy.special.expit(x)
        
        pass

    
    def train(self, inputs_list, targets_list):
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        hidden_inputs = numpy.dot(self.wih, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        final_inputs = numpy.dot(self.who, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)
        output_errors = targets - final_outputs
        hidden_errors = numpy.dot(self.who.T, output_errors) 
        self.who += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs))
        self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))
        pass


    def query(self, inputs_list):

        inputs = numpy.array(inputs_list, ndmin=2).T
        hidden_inputs = numpy.dot(self.wih, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        final_inputs = numpy.dot(self.who, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)
        return final_outputs

#### 模型的搭建与训练

首先，我们需要定义神经网络的输入层，隐藏层，输出层的节点个数，学习率等。并且利用这些参数，初始化一个神经网络实列。代码如下：

In [ ]:
# 输入层，隐藏层，输出层的节点个数
input_nodes = 784 #输入层节点选择 784 个的原因是因为一张手写数字图像的大小为 28*28=784
hidden_nodes = 100 
output_nodes = 10 #最后的答案是 0-9 十个数字

# 学习率
learning_rate = 0.1

# 创建一个神经网络实例
n = neuralNetwork(input_nodes,hidden_nodes,output_nodes, learning_rate)
n

选择使用 100 个隐藏层节点并不是通过使用科学的方法得到的。我们认为，神经网络应该可以发现在输入中的特征或模式，这些模式或特征可以使用比输入本身更简短的形式表达，因此没有选择比 784 大的数字。通过选择使用比输入节点的数量小的值，强制网络尝试总结输入的主要特点。但是，如果选择太少的隐藏层节点，那么就限制了网络的能力，使网络难以找到足够的特征或模式，也就会剥夺神经网络表达其对 MNIST 数据理解的能力。给定的输出层需要 10 个标签，对应于 10 个输出层节点，因此，选择 100 这个中间值作为中间隐藏层的节点数量，似乎有点道理。

接下来，让我们加载训练神经网络所需要的 MNIST 数据：

In [ ]:
# 将 MNIST 训练数据集加载进一个 list 集合中
training_data_file = open("mnist_train_100.csv", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()
training_data_file

接下来，利用循环，对 MNIST 中的每条数据的输入输出都进行上述的预处理操作。最后将处理后的数据放入神经网络中进行训练。

In [ ]:
#训练神经网络

index = 0 #记录当前训练的是第几条
# 遍历所有的训练数据
for record in training_data_list:
    # 通过逗号分割，将一条条字符串（从文件中读出来的数据，每条其实是一个字符串），分割成数组
    all_values = record.split(',')
    # 将输入的范围置为 0.01 - 1.0
    inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    # 将输出置为上面的那种 10 个节点的固定形式
    targets = numpy.zeros(output_nodes) + 0.01
    targets[int(all_values[0])] = 0.99
    
    #开始训练
    n.train(inputs, targets)
    print("已训练完第",index,"条数据")
    index = index+1 
    pass 

#### 模型的测试

现在，我们至少已经使用了一个较小的 100 条记录的子集来训练网络，我们希望测试训练效果如何。使用称为测试数据集的第二个数据集来测试神经网络。

首先需要获得测试记录，这与用于获取训练数据的 Python 代码非常相似。 

In [ ]:
# 加载测试数据
test_data_file = open("mnist_test_10.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close() 
# 取出第一条数据
all_values = test_data_list[0].split(',')
# 查看这条数据的真实标签
print(all_values[0])

让我们把这条数据放入模型中，查看预测的结果：

In [ ]:
n.query((numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01)

通过结果可以看到，输出的 10 个节点中，只有第 7 个节点被激活。则说明神经网络对该张图像的预测为 7。预测结果，准确。

我们训练了神经网络，让神经网络告诉我们图片中所代表的数字是什么。请记住，神经网络之前没有见过那张图片，它不是训练数据集的一部分。因此，神经网络能够正确区分它从来没有见过的手写字符。这真是让人印象深刻啊！

只需几行简单的 Python，我们就已经创建了一个神经网络，这个神经网络可以执行许多人认为是具备人工智能的事情——它学会了识别人的笔迹图片。

更令人称奇的是，我们只是使用完整的训练数据集的一个小子集对神经网络进行了训练。请记住，训练数据集有 60 000 条记录，我们只训练了 100 条记录。我曾经认为这不能成功！让我们扯满篷帆，继续前进，编写代码来看看神经网络对测试数据集的其余记录有何表现。在此过程中，我们可以记录分数，这样迟些时候，再看看改进神经网络学习能力的想法是否能够成功，同时也可以比较一下其他神经网络的表现如何。

In [ ]:
# scorecard for how well the network performs, initially empty
scorecard = []

# 遍历所有的测试数据
for record in test_data_list:
    # 通过逗号分割，将一条条字符串（从文件中读出来的数据，每条其实是一个字符串），分割成数组
    all_values = record.split(',')
    #得到每张图片的正确标签
    correct_label = int(all_values[0])
    print(correct_label, "correct label")
    
    #对输入进行预处理
    inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    
    # 利用训练好的神经网络，对输入进行预测
    outputs = n.query(inputs)
    
    # 该函数就是找到输出的10个节点中，值最大的（受激活）的节点的下标
    #即得到预测的标签值
    label = numpy.argmax(outputs)
    print(label, "network's answer")
    # 判断预测标签值和真实标签值是否相同
    #如果相同，则在列表中添加一个 1
    if (label == correct_label):
        scorecard.append(1)
        
    #如果不相同，则在列表中添加一个 0
    else:
        scorecard.append(0)
        pass
    pass

上述代码中，我们利用 scorecard 变量来进行计分。我们可以对他进行求和，来计算整个测试数据集中预测正确的数据集个数。

In [ ]:
# 计算正确记录数
#将 scorecard 变量转为 numpy 类型，方便求和
scorecard_array = numpy.asarray(scorecard)
#正确的条数/总条数 = 模型的正确率 
print ("performance = ", scorecard_array.sum() /scorecard_array.size) 

正如我们预期的，这段代码生成了分数 0.6，即 60% 的准确率。

#### 使用完整数据集进行训练和测试

之前，我们只是使用了 MNIST 中的 100 条数据进行训练。接下来，我们使用全部数据集，进行训练。


此时改变文件名，这样就可以指向具有 60 000 条记录的完整的训练数据集，以及具有 10 000 条记录的测试数据集。

首先读取数据，并训练，其实代码和上面的代码一样，只是换了文件名而已。由于样本数众多，因此请耐心等待（大概需要3~5min）。

In [ ]:
#训练神经网络
# 加载训练数据，此时用的是，全部的数据
training_data_file = open("mnist_train.csv", 'r')
#其他代码，同上
training_data_list = training_data_file.readlines()
training_data_file.close() 

print("开始训练，请耐心等待( 3~5 min)....")
for record in training_data_list:
    all_values = record.split(',')
    inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    targets = numpy.zeros(output_nodes) + 0.01
    targets[int(all_values[0])] = 0.99
    n.train(inputs, targets)
    pass
print("训练完毕")

同样，我们使用 10 000 条完整的数据集对网络进行测试，得到模型的准确率。

In [ ]:
# 完整数据的测试代码和部分数据的测试代码相同，只需要
test_data_file = open("mnist_test.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close() 
scorecard = []
print("开始遍历测试数据，请耐心等待（大概1~2min）。。。")
for record in test_data_list:
    all_values = record.split(',')
    correct_label = int(all_values[0])
    inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    outputs = n.query(inputs)
    label = numpy.argmax(outputs)
    if (label == correct_label):
        scorecard.append(1)
    else:
        scorecard.append(0)
        pass
    pass
scorecard_array = numpy.asarray(scorecard)
print ( "模型的准确率为：", scorecard_array.sum() /scorecard_array.size) 

这个略低于 95% 的准确性(若你得到正确率过低，可多次运行上上个代码块中的训练函数)，可以与记录在 [官方网页](http://yann.lecun.com/exdb/mnist/) 的行业标准媲美。我们可以看到，比起一些历史基准，这个准确率还是略胜一筹的，这里列出的最简单的神经网络方法所表现的准确率为 95.3% ，而我们的神经网络的性能大致相当。 因此，足够多的训练数据，是可以提高模型的正确率的。

#### 自己的手写数字

我们一直使用来自 MNIST 数据集的数字图片。为什么不使用自己的笔迹呢？ 

在这个实验中，我们将使用自己的笔迹创建测试数据集。我们也将尝试使用不同的书写风格，使用嘈杂或抖动的图片，来观察神经网络的应对能力如何。

下面是我制作的一些图片：

<img  width="300px" src="https://doc.shiyanlou.com/courses/uid1166617-20191228-1577512331201/wm">

让我们来加载这些图片：

In [ ]:
!wget -nc https://labfile.oss.aliyuncs.com/courses/1500/my_own_images.zip
!unzip -o "my_own_images.zip" 

数字 5 就是我的笔迹。数字 4 是用粉笔而不是马克笔写的。数字 3 是我的笔迹并有意切成一段一段的。数字 2 是传统的报纸或书籍字体，但是进行了模糊处理。数字 6 有意做成抖动的样子，好像是在水中的倒影。最后一张图片与前面的数字相同，但是添加了噪声，来看看我们是否可以增加神经网络的工作难度。 

虽然这很有趣，但是这里蕴含了很严肃的一点。人类大脑在遭受损害后，其能力依然能够得到良好发挥，科学家对此深感震惊。这暗示着，神经网络将它们所学到的知识分布在几条链接权重上，也就是说，如果若干链接权重遭受了一定损害，神经网络也可以表现得相当好。这同时意味着，如果输入图像被损坏或不完整，神经网络也可以表现得相当好。这是一种很强大的功能，这就是我们希望用上图中断断续续的 3 进行测试的能力。 

我们需要创建较小的 PNG 图片，将它们调整到 $28 \times 28$，这样就可以匹配曾经用过的来自 MNIST 数据集的图片。你可以使用图像编辑器做到这一点。

Python 库再次帮助了我们，它从常见的图像文件格式中（包括 PNG 格式）读取和解码数据。看看下面这段简单的代码：

In [ ]:
#这里只测试数字 2 的图片，其他图片测试方法相同
import imageio
#加载图片
img_array = imageio.imread("2828_my_own_2.png",as_gray=True)
img_data = 255.0 - img_array.reshape(784)
img_data = (img_data / 255.0 * 0.99 ) + 0.01
img_data

imageio.imread() 函数帮助我们从图像文件，如 PNG 或 JPG 文件中，读取数据。必须导入 imageio 库来使用这个函数。参数 “as_gray= True” 将图像变成简单的浮点数数组，如果图像是彩色的，那么颜色值将被转换为所需要的灰度。 

下一行代码重塑数组，将其从 28×28 的方块数组变成很长的一串数值，这是我们需要馈送给神经网络的数据。此前，我们已经多次进行这样的操作了。但是，这里比较新鲜的一点是从 255.0 中减去了数组的值。这样做的原因是，常规而言，0 指的是黑色，255 指的是白色，但是，MNIST 数据集使用相反的方式表示，因此不得不将值逆转过来以匹配 MNIST 数据。

最后一行代码是我们很熟悉的，它将数据值进行缩放，使得它们的范围变成0.01到1.0。 演示读取 PNG 文件的 [示例代码](https://github.com/makeyourownneuralnetwork/makeyourownneuralnetwork/blob/master/part3_load_own_images.ipynb) 可以在GitHub上找到。

接下来，让我们利用以及训练好的神经网络，对我们自己写的字符进行预测：

In [ ]:
# 将数据传入神经网络并输出
outputs = n.query(img_data)
img_data_label = numpy.argmax(outputs)
print("实际标签为 2 ，预测标签为:",img_data_label)

这样做成功了吗？当然成功了。下图总结使用我们自己制作的图像查询的结果。

<img width="400px" src="https://doc.shiyanlou.com/courses/uid1166617-20191228-1577514520501/wm">

可以看到，神经网络能够识别我们创建的所有图像，包括有意损坏的数字“3”。只有在识别添加了噪声的数字“6”时失败了。

使用你自己的图像，尤其是手写的图像试试看，证明你的神经网络确实能够工作。

并且，仔细观察，要将图像损坏或变形到什么程度，神经网络才会失败。神经网络的弹性将会给你留下深刻的印象。

### 模型的优化

#### 学习率的调整

我们可以尝试的第二个改进是调整学习率。先前没有真正使用不同的值进行实验，就将它设置为 0.3 了。

试一下将学习率翻倍，设置为 0.6 ，看看提高学习率对整个网络的学习能力是否有益。如果此时运行代码，会得到 0.9047 性能得分。这比以前更糟。因此，看起来好像大的学习率导致了在梯度下降过程中有一些来回跳动和超调。

使用 0.1 的学习率再试一次。这次，性能有所改善，得到了 0.9523 分。在性能上，这与网站上列出的具有 1 000 个隐藏层节点的神经网络类似。因此，我们“以少胜多”了。 

如果继续设置一个更小的 0.01 学习率，会发生什么情况？性能没有变得更好，得分为 0.9241 。因此，似乎过小的学习率也是有害的。

由于限制了梯度下降发生的速度，使用的步长太小了，因此对性能造成了损害，这个结果也是有道理的。 

下图画出了这些结果。我们应该多次进行了这些实验，减小随机性以及在梯度下降过程中不好的路径带来的影响，只有这样的方法才是科学的，但是这依然能够有助于我们明白一个总体思路，那就是对于学习率存在最佳点。

<img width="400px" src="https://doc.shiyanlou.com/courses/uid1166617-20191227-1577419420751/wm">

上图表明，学习率在 0.1 和 0.3 之间可能会有较好的表现，因此，尝试 0.2 的学习率，得到 0.9537 的性能得分。比起 0.1 或 0.3 ，这个表现确实好了一些。我们可以绘制图表，对所发生的事情得到一种较好的认识，在其他情况下，你也应该考虑这种方法——和一串数字相比，图表有助于更好地理解！因此，我们将坚持使用 0.2 的学习率，这看起来似乎是 MNIST 数据集和神经网络的甜蜜点

顺便说一句，由于代码运行的整个过程有一点随机，因此，如果你自己运行这段代码，成绩会略有不同。你的初始随机权重可能不同于我的初始随机权重，因此你的代码与我的代码所使用的梯度下降路线有所不同。

#### 多次运行

接下来可以做的改进，是使用数据集，重复多次进行训练。

有些人把训练一次称为一个世代。因此，具有 10 个世代的训练，意味着使用整个训练数据集运行程序 10 次。为什么要这么做呢？特别是，如果这次计算机花的时间增加到 10 或 2 0甚至 30 分钟呢？这是值得的，原因是通过提供更多爬下斜坡的机会，有助于在梯度下降过程中进行权重更新。

试一下使用 2 个世代。由于现在我们在训练代码外围添加了额外的循环，因此代码稍有改变。下面的代码显示了外围循环，将代码着色有助于看到发生了什么（需要等待8~10 min，建议直接点击左上角的方形按钮，直接停止）。

In [ ]:
#其他代码不变，只是在原来的基础上，添加一个外循环
#epochs 存的就是需要循环的世代的次数
epochs = 2 
for e in range(epochs):
    for record in training_data_list:
        all_values = record.split(',')
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) +0.01
        targets = numpy.zeros(output_nodes) + 0.01
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)
        pass
    pass

使用 2 个世代神经网络所得到的性能得分为 0.9579 ，比只有 1 个世代的神经网络有所改进。

就像调整学习率一样，让我们使用几个不同的世代进行实验并绘图，以可视化这些效果。直觉告诉我们，所做的训练越多，所得到的性能越好。有人可能会注意到，太多的训练实际上会过犹不及，这是由于网络过度拟合训练数据，因此网络在先前没有见到过的新数据上表现不佳。不仅是神经网络，在各种类型的机器学习中，这种过度拟合也是需要注意的。

发生的事情如下所示： 

<img width="300px" src="https://doc.shiyanlou.com/courses/uid1166617-20191227-1577420550767/wm">

结果呈现出不可预测性。在大约 5 或 7 个世代时，有一个甜蜜点。在此之后，性能会下降，这可能是过度拟合的效果。性能在 6 个世代的情况下下降，这可能是运行中出了问题，导致网络在梯度下降过程中被卡在了一个局部的最小值中。事实上，由于没有对每个数据点进行多次实验，无法减小随机过程的影响，因此我们已经预见到结果会有各种变化。这就是为什么保留了 6 个世代这个奇怪的点，这是为了提醒我们，神经网络的学习过程其核心是随机过程，有时候工作得不错，有时候工作得很糟。

另一个可能的原因是，在较大数目的世代情况下，学习率可能设置过高了。继续这个实验，将学习率从 0.2 减小到 0.1 ，看看会发生什么情况。

在 7 个世代的情况下，峰值性能高达 0.9628 或 96.28%。

下图显示了在学习率为 0.1 情况下，得到的新性能与前一幅图叠加的情况。 

<img width="300px" src="https://doc.shiyanlou.com/courses/uid1166617-20191227-1577421000597/wm">

可以看到，在更多世代的情况下，减小学习率确实能够得到更好的性能。0.9689 的峰值表示误差率接近 3% ，这可以与 Yann LeCun 网站上的神经网络标准相媲美了。

直观上，如果你打算使用更长的时间（多个世代）探索梯度下降，那么你可以承受采用较短的步长（学习率），并且在总体上可以找到更好的路径，这是有道理的。确实，对于 MNIST 学习任务，我们的神经网络的甜蜜点看起来是 5 个世代。请再次记住，我们在使用一种相当不科学的方式来进行实验。要正确、科学地做到这一点，就必须为每个学习率和世代组合进行多次实验，尽量减少在梯度下降过程中随机性的影响。 

#### 改变网络形状 

我们还没有尝试过改变神经网络的形状，也许应该更早尝试这件事。让我们试着改变中间隐藏层节点的数目。一直以来，我们将它们设置为 100 ！ 

在尝试使用不同数目的隐藏层节点进行实验之前，让我们思考一下，如果这样做可能会发生什么情况。隐藏层是发生学习过程的层次。请记住，输入节点只需引入输入信号，输出节点只要送出神经网络的答案，是隐藏层（可以多层）进行学习，将输入转变为答案。这是学习发生的场所。事实上，隐藏层节点前后的链接权重具有学习能力。 

如果隐藏层节点太少，比如说 3 个，那么你可以想象，这不可能有足够的空间让网络学习任何知识，并将所有输入转换为正确的输出。这就像要 5 座车去载 10 个人。你不可能将那么多人塞进去。计算机科学家称这种限制为学习容量。虽然学习能力不可能超过学习容量，但是可以通过改变车辆或网络形状来增加容量。 

如果有 10 000 个隐藏层节点，会发生什么情况呢？虽然我们不会缺少学习容量，但是由于目前有太多的路径供学习选择，因此可能难以训练网络。这也许需要使用 10 000 个世代来训练这样的网络。 

让我们进行一些实验，看看会发生什么情况。

<img width="300px" src="https://doc.shiyanlou.com/courses/uid1166617-20191227-1577421857403/wm">


可以看到，比起较多的隐藏层节点，隐藏层节点数量少，其效果不是很理想，这是我们预期的结果。但是，只有 5 个隐藏层节点的神经网络，其性能得分就可以达到 0.7001 ，鉴于只给了如此少的学习场所，而网络仍有 70% 的正确率，这已经相当惊人了。 

请记住，迄今为止，程序运行的是 100 个隐藏层节点。只用 10 个隐藏层节点，网络就得到了0.8998 的准确性，这同样让人侧目。只使用我们曾经用过的节点数目的 1/10 ，网络的性能就跳到90% 。只使用如此少的隐藏层节点或学习场所，神经网络就能够得到如此好的结果。这也证明了神经网络的力量。这一点值得我们赞赏。

随着增加隐藏层节点的数量，结果有所改善，但是不显著。由于增加一个隐藏层节点意味着增加了到前后层的每个节点的新网络链接，这一切都会产生额外较多的计算，因此训练网络所用的时间也显著增加了！因此，必须在可容忍的运行时间内选择某个数目的隐藏层节点。

对于我的计算机而言，这个数字是 200 个节点。你的计算机可能会相对较快或相对较慢。 

我们还创造了准确度的新纪录，使用 200 个节点，得分 0.9751 。使用 500 个节点，运行较长的时间，我们的神经网络得到了 0.9762 分。相比于 Yann LeCun 的网站上列出的基准，这是相当不错的成绩了。

回过头去，看看以前的图，可以发现，通过改变网络形状，先前约 95% 的准确度这个“冥顽不灵”的极限已经被打破了。

### 实验总结

回顾这项工作，我们只用先前，介绍的简单概念以及简单的 Python 代码，创建了一个神经网络。在此过程中，没有任何多余花哨、神奇的数学，神经网络就已经表现得如此出众，相比于学者和研究人员所编写的神经网络，这个神经网络的表现也是可圈可点。

<hr><div style="color: #999; font-size: 12px;"><i class="fa fa-copyright" aria-hidden="true"> 本课程内容版权归实验楼所有，禁止转载、下载及非法传播。</i></div>